### datarobot api demo notebook: 
## remove collinearity from your model and get clearer feature impact

###### v1.1

What we're going to do here:

1. Build your project the usual way, including all your collinear features.  

2. Identify your best-performing model here (for speed later on, use the best-performing non-blender model if there’s not much difference in out-of-sample performance between that and the best blender)

3. Use your domain expertise/feature impact to identify a list of the features you want to investigate for multicollinearity, and a list of the other features you don’t want to investigate (let’s call the latter list the ‘stub’)

4. Using the API, build multiple feature lists, each containing all ‘stub’ features and one of the features to be investigated

5. Using the API, re-train your best model from (2) on each of the feature lists from (4) and calculate its feature impact

6. For all models from (5), compare i. out-of-sample performance (ideally cross-validated) and ii. the un-normalised impact of each feature being investigated.  The model with the best o-o-s performance should be associated with the ‘best’ feature (i.e. the one collinear feature to use as a ‘base'); cross-check this with the feature impacts to make sure that that feature also has the biggest un-normalised impact.  

7. Feature engineering time: calculate the differences between the other collinear features and the ‘base’ feature you will use (the ‘differenced’ features). 

8. Re-run Autopilot on a feature list containing the ‘stub’ features, the ‘base’ (strongest) collinear feature and the ‘differenced’ feature.

9. Go and make yourself a nice cup of coffee while the resulting API code runs...


Let's start with some standard imports, credentials etc.

In [1]:
# standard imports
import os
import numpy as np
import pandas as pd
import datarobot as dr

# your credentials here :)
API_TOKEN = ""  ## <------ POPULATE THIS!
API_ENDPOINT = "https://app.eu.datarobot.com/api/v2"
dr.Client(endpoint=API_ENDPOINT, token=API_TOKEN)

We'll set some constants

In [2]:
# CONSTANTS - path, data file name
DATA_PATH = "/Users/Peter/Documents/DataRobot"
TARGET_FEAT = "SalePrice"
CREATE_PROJ_NAME = "ZPCN - collinear feature impact demo"
TRAIN_DATA = "ZPCN.csv"

FEATLIST_STEM = 'zcoll_'
SCORING_TYPE = 'crossValidation'

# Which project to use
GET_PROJECT_ID = "5bafc6c4db9d452995b20585"
# GET_PROJECT_ID = None  # build project from scratch

# DR settings
MAX_WORKERS = 20  # set your max number of workers here...

Now define a couple of functions to improve on the error handling in the out-of-the-box API.

In [3]:
# -------------------------------------------------------------------------------------------------
# FUNCTION DEFINITIONS —
# GET VARIOUS THINGS WITH ERROR HANDLING
# -------------------------------------------------------------------------------------------------


# feature impact - get if already computed, add if not
def get_or_calc_feat_impact(model):
    try:
        # if it's computed already
        fi = model.get_feature_impact()
        print('Retrieved feature impact for', model.id, model.model_type)
    except dr.errors.ClientError as e:
        # the feature impact score haven't been computed yet, so compute it
        assert e.status_code == 404  # check there's nothing else kaput
        print('Computing feature impact for', model.id, model.model_type, '...')
        impact_job = model.request_feature_impact()
        fi = impact_job.get_result_when_complete()
        print('...retrieved.')

    return fi


# feature list - get if already added, add if not
def get_or_create_featurelist(project, fl_name, features):
    try:
        flist = dr_proj.create_featurelist(fl_name, features)
    except dr.errors.ClientError as e:
        assert e.status_code == 422  # check there's nothing else kaput
        # this is horrible syntax, but works — we can't access featurelists by name, so we iterate
        # over all fl in the project until we find one that matches the name of what we want
        scratch_fl = project.get_featurelists()
        flist = [fl for fl in scratch_fl if fl.name == fl_name][0]
    return flist



Build your project the usual way, including all your collinear features.  Or get one you made earlier — but either way, you'll need the source data (at present, we can't get the training data out of the model), so let's load that first.

In [4]:
# -------------------------------------------------------------------------------------------------
# MAIN
# -------------------------------------------------------------------------------------------------

# we'll need the data either way...
print('Reading data...')
model_data = pd.read_csv(os.path.join(DATA_PATH, TRAIN_DATA), encoding='latin-1')

# build project if don't already have
if GET_PROJECT_ID is None:
    print('No project ID given.  Creating project...')
    dr_proj = dr.Project.create(model_data, project_name=CREATE_PROJ_NAME)

    print('Setting target variable...')
    dr_proj.set_worker_count(MAX_WORKERS)
    dr_proj.set_target(target=TARGET_FEAT)

    print('Running autopilot...')
    dr_proj.wait_for_autopilot()

else:
    # get existing project
    print('Getting project', GET_PROJECT_ID)
    dr_proj = dr.Project.get(project_id=GET_PROJECT_ID)

Reading data...
Getting project 5bafc6c4db9d452995b20585


Identify your best-performing model here (for speed later on, I’d suggest using the best-performing non-blender model if there’s not much difference in out-of-sample performance between that and the best blender)

In [5]:
# get the best non-blender model
proj_metric = dr_proj.metric
models = dr_proj.get_models(order_by=['-metric', 'sample_pct'])
# models = dr_proj.get_models()
for m in models:
    if u'Blender' not in m.model_type:
        best_model = m
        break

# get blueprint, featurelist associated with best model
best_bp = m.blueprint_id
best_fl_id = m.featurelist_id
best_fl = dr.Featurelist.get(dr_proj.id, best_fl_id)

best_model

Model('Light Gradient Boosting on ElasticNet Predictions (Gamma Loss)')

We want a list of the features used in our best model, so that we can remove our collinear features later.  

In [6]:
# get list of features used
feat_list = best_model.get_features_used()
feat_list

['1stFlrSF',
 '2ndFlrSF',
 '3SsnPorch',
 'Alley',
 'BedroomAbvGr',
 'BldgType',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtQual',
 'BsmtUnfSF',
 'CentralAir',
 'Condition1',
 'Condition2',
 'Electrical',
 'EnclosedPorch',
 'ExterCond',
 'ExterQual',
 'Exterior1st',
 'Exterior2nd',
 'Fence',
 'FireplaceQu',
 'Fireplaces',
 'Foundation',
 'FullBath',
 'Functional',
 'GarageArea',
 'GarageCars',
 'GarageCond',
 'GarageFinish',
 'GarageQual',
 'GarageType',
 'GarageYrBlt',
 'GrLivArea',
 'HalfBath',
 'Heating',
 'HeatingQC',
 'HouseStyle',
 'Id',
 'KitchenAbvGr',
 'KitchenQual',
 'LandContour',
 'LandSlope',
 'LotArea',
 'LotAreaNoise1',
 'LotAreaNoise2',
 'LotAreaNoise3',
 'LotConfig',
 'LotFrontage',
 'LotShape',
 'LowQualFinSF',
 'MSSubClass',
 'MSZoning',
 'MasVnrArea',
 'MasVnrType',
 'MiscFeature',
 'MiscVal',
 'MoSold',
 'Neighborhood',
 'OpenPorchSF',
 'OverallCond',
 'OverallQual',
 'PavedDr

We're going to decide which features we want to investigate for collinearity by hand here.  You could equally put a routine in here to identify clusters of correlated features and work through them, but the idea is the same.

In [7]:
# which features do we want to investigate for being the most impactful?
coll_feats = [u'LotArea',
              u'LotAreaNoise1',
              u'LotAreaNoise2',
              u'LotAreaNoise3',
              ]

Now that that's decided, let's get a 'stub' (rump) feature list which includes all model features other than the above.

In [8]:
# get the stub feature list excluding the above
stub_feats = list(set(feat_list) - set(coll_feats))
stub_feats

['Exterior2nd',
 'FireplaceQu',
 'OverallQual',
 'Exterior1st',
 'GrLivArea',
 'LotFrontage',
 'Electrical',
 'LandContour',
 'MoSold',
 'GarageCond',
 'Alley',
 'Neighborhood',
 'Id',
 'FullBath',
 'GarageType',
 'BedroomAbvGr',
 'EnclosedPorch',
 '2ndFlrSF',
 'TotRmsAbvGrd',
 'SaleType',
 'OverallCond',
 'BsmtHalfBath',
 'BsmtFinSF1',
 'CentralAir',
 'BsmtFinSF2',
 'YearBuilt',
 'LotConfig',
 'Fence',
 'Condition2',
 'ScreenPorch',
 'ExterCond',
 'MSZoning',
 'MasVnrArea',
 'MiscVal',
 'RoofMatl',
 'BsmtFinType1',
 'TotalBsmtSF',
 'OpenPorchSF',
 'HalfBath',
 'LotShape',
 'MiscFeature',
 'PoolQC',
 'GarageCars',
 'Heating',
 'KitchenQual',
 'GarageFinish',
 'BsmtFinType2',
 'Condition1',
 'BsmtQual',
 'GarageArea',
 'YrSold',
 'GarageYrBlt',
 '3SsnPorch',
 'LandSlope',
 'WoodDeckSF',
 'PavedDrive',
 'Fireplaces',
 'GarageQual',
 'MasVnrType',
 'PoolArea',
 'BsmtCond',
 'BsmtExposure',
 'HouseStyle',
 'LowQualFinSF',
 'Functional',
 'Street',
 'HeatingQC',
 'YearRemodAdd',
 'Foundatio

We can now calculate the feature impact for our baseline model using the function we defined earlier, which checks whether we've already run feature impact, and allows for that.

In [9]:
# get baseline feature impact and format as a DF
fi = get_or_calc_feat_impact(best_model)
base_fi = pd.DataFrame(fi).set_index('featureName')


Retrieved feature impact for 5bafc6f457d023e7ee996374 Light Gradient Boosting on ElasticNet Predictions (Gamma Loss)


We're going to build separate models for each feature in our list of collinear features — each model will include that feature, plus the stub.  Let's init a few dicts to store various things relating to these models.

In [10]:
# and we'll store a few things in dicts, starting with our baseline
impacts = {'baseline': base_fi}
retrained_models = {'baseline': best_model}
# jobs = {'baseline': None}

feat_lists = {'baseline': get_or_create_featurelist(dr_proj, 'baseline', feat_list)}
model_ids = {'baseline': best_model.id}
job_ids = {}
fi_jobs = {}

# and store the unnormalized impacts in a df for later comparison
unn_imps = pd.DataFrame(base_fi.impactUnnormalized).rename(
    columns={'impactUnnormalized': 'baseline'})

Right, we're ready to build some more models.  We'll kick off the model re-trains first: we are re-training the best model with various feature lists; this will give us the same number of new models as there are features we're interested in.

Once the model building jobs are running, we can retrieve the actual Model objects and then cross-validate them.  We'll also calculate feature impact for each of the new models.  This will take a little while to run.

After that, we compile a dataframe showing all the un-normalized feature impacts, and examine the FIs for the impacts of each individual variable; this gives us a much better idea of which of the variables is most 'impactful'.

*For efficiency, these tasks should be run in close sequence, and refreshed models already built with these feature lists should be deleted prior to re-running the cell below.*

In [11]:
# Now we'll work through the individual features - first, build the models
for f in coll_feats:
    # Make a feature list of stub + f
    featl_name = FEATLIST_STEM + f
    # check feat list hasn't been created yet --> create
    feat_lists[f] = get_or_create_featurelist(dr_proj, featl_name, stub_feats + [f])
    # train the model
    # try:
    job_ids[f] = best_model.train(featurelist_id=feat_lists[f].id,
                                  scoring_type=SCORING_TYPE)
    retrained_models[f] = None
    # except dr.errors.JobAlreadyRequested as e:
    # assert e.status_code == 422
    # retrained_models[f] =
    # model_ids[f] = job_ids[f]

# then, build the feature impacts once model created (it's faster to split this way)
for f in coll_feats:
    # get the feature list name
    featl_name = FEATLIST_STEM + f
    # and the model
    retrained_models[f] = dr.models.modeljob.wait_for_async_model_creation(project_id=dr_proj.id,
                                                                           model_job_id=job_ids[f])
    print('Retrained model on list', featl_name, 'as Model.id', retrained_models[f].id)
    # if feat impact is calculated already, get it
    try:
        impacts[f] = retrained_models[f].get_feature_impact()
        fi_jobs[f] = None
    except dr.errors.ClientError as e:
        # the feature impact score haven't been computed yet, so compute it
        impacts[f] = None
        assert e.status_code == 404  # check there's nothing else kaput
        print('Computing feature impact for', retrained_models[f].id,
              retrained_models[f].model_type, '...')
        fi_jobs[f] = retrained_models[f].request_feature_impact()

# finally, mop up the feature impact jobs that are being calculated once complete
# and compile the unnormalized feature impacts into one DF
for f in coll_feats:
    # retrieve fi if not already calculated
    if impacts[f] is None:
        impacts[f] = fi_jobs[f].get_result_when_complete()
    # translate fi into a dataf
    impacts[f] = pd.DataFrame(impacts[f]).set_index('featureName')
    featl_name = FEATLIST_STEM + f
    unn_imps[featl_name] = impacts[f].impactUnnormalized

Retrained model on list zcoll_LotArea as Model.id 5bafe10edb9d45293ab20685
Computing feature impact for 5bafe10edb9d45293ab20685 Light Gradient Boosting on ElasticNet Predictions (Gamma Loss) ...
Retrained model on list zcoll_LotAreaNoise1 as Model.id 5bafe114db9d45293ab20698
Computing feature impact for 5bafe114db9d45293ab20698 Light Gradient Boosting on ElasticNet Predictions (Gamma Loss) ...
Retrained model on list zcoll_LotAreaNoise2 as Model.id 5bafe115db9d4529f5b2063c
Computing feature impact for 5bafe115db9d4529f5b2063c Light Gradient Boosting on ElasticNet Predictions (Gamma Loss) ...
Retrained model on list zcoll_LotAreaNoise3 as Model.id 5bafe118db9d45293ab206ac
Computing feature impact for 5bafe118db9d45293ab206ac Light Gradient Boosting on ElasticNet Predictions (Gamma Loss) ...


Having compiled the un-normalized feature impacts into one dataframe, we can compare them.

In [12]:
unn_imps.loc[coll_feats, :]

,baseline,zcoll_LotArea,zcoll_LotAreaNoise1,zcoll_LotAreaNoise2,zcoll_LotAreaNoise3
featureName,,,,,
LotArea,0.000291,0.001424,NaN,NaN,NaN
LotAreaNoise1,0.000192,NaN,0.001262,NaN,NaN
LotAreaNoise2,0.000258,NaN,NaN,0.001319,NaN
LotAreaNoise3,0.000057,NaN,NaN,NaN,0.001332


Which is the most impactful feature, from the *Feature Impact* calculations?

In [13]:
# let's look for the most impactful feature now -- this will be on a sample of 2,500 rows as it's from F.I.
most_impactful_FI = unn_imps.loc[coll_feats, :].max().idxmax()
most_impactful_FI

'zcoll_LotArea'

We should also look at this at the model level — comparing the accuracy metrics for each of our new models will give us an even better idea of which of our features is most impactful over the entire model rather than a sample (*Feature Impact* calculations are done for a sample of 2,500 rows max.)

In [14]:
# get a DF of the model metrics
optimisation_metric = dr_proj.metric

metrics = {}

for rmk in retrained_models.keys():
    metrics[rmk] = {mname: retrained_models[rmk].metrics[mname][SCORING_TYPE]
                    for mname in retrained_models[rmk].metrics.keys()}

metr_DF = pd.DataFrame(metrics)
metr_DF

,LotArea,LotAreaNoise1,LotAreaNoise2,LotAreaNoise3,baseline
FVE Gamma,0.910052,0.910178,0.910712,0.910500,0.910696
FVE Poisson,0.900766,0.900724,0.901366,0.900922,0.901886
FVE Tweedie,0.908112,0.908150,0.908716,0.908382,0.908934
Gamma Deviance,0.014458,0.014460,0.014364,0.014408,0.014358
Gini Norm,0.966126,0.966090,0.966318,0.966178,0.966356
MAE,15486.097642,15495.193540,15435.069278,15468.165558,15389.767942
MAPE,8.598070,8.582134,8.566762,8.568354,8.542974
Poisson Deviance,3079.152472,3086.371556,3063.163066,3080.238534,3048.175322
R Squared,0.867238,0.867152,0.868038,0.867448,0.869656
RMSE,29015.553326,29034.864840,28930.361220,28997.120862,28807.181234


<pre>best_performing_feature_on_single_metric</pre> is a horrible variable name, so let's call it <pre>bpfsm</pre> instead.

In [15]:
# let's cross-check the most impactful feature -- this time on the full data
# (we want the feature which gives us the best-performing model stand-alone)
USE_MIN = True

if USE_MIN:
    bpfsm = metr_DF.loc[optimisation_metric, coll_feats].idxmin()
else:
    bpfsm = metr_DF.loc[optimisation_metric, coll_feats].idxmax()
    
bpfsm

'LotAreaNoise2'

Now we can difference the other collinear features against the best-performing one...

In [16]:
# now let's difference the other features 
USE_RATIO = False

new_model_data = model_data.loc[:, stub_feats + [bpfsm]]
for f in coll_feats:
    if f != bpfsm:
        if USE_RATIO:
            new_model_data['r_' + f + '_' + bpfsm] = model_data[f] / new_model_data[bpfsm]
        else:
            new_model_data['d_' + f + '_' + bpfsm] = model_data[f] - new_model_data[bpfsm]

new_model_data.head(10)

,Exterior2nd,FireplaceQu,OverallQual,Exterior1st,GrLivArea,LotFrontage,Electrical,LandContour,MoSold,GarageCond,...,1stFlrSF,SalePrice,BsmtUnfSF,BldgType,RoofStyle,MSSubClass,LotAreaNoise2,d_LotArea_LotAreaNoise2,d_LotAreaNoise1_LotAreaNoise2,d_LotAreaNoise3_LotAreaNoise2
0,VinylSd,NaN,7,VinylSd,1710,65.0,SBrkr,Lvl,2,TA,...,856,208500,150,1Fam,Gable,60,8304.672648,145.327352,11.902487,-54.262696
1,MetalSd,TA,6,MetalSd,1262,80.0,SBrkr,Lvl,5,TA,...,1262,181500,284,1Fam,Gable,20,9463.521549,136.478451,188.258521,337.678959
2,VinylSd,TA,7,VinylSd,1786,68.0,SBrkr,Lvl,9,TA,...,920,223500,434,1Fam,Gable,60,11166.032010,83.967990,222.376150,167.045990
3,Wd Shng,Gd,7,Wd Sdng,1717,60.0,SBrkr,Lvl,2,TA,...,961,140000,540,1Fam,Gable,70,9332.794510,217.205490,179.872760,255.745391
4,VinylSd,TA,8,VinylSd,2198,84.0,SBrkr,Lvl,12,TA,...,1145,250000,490,1Fam,Gable,60,14379.518330,-119.518330,-120.987350,-31.636360
5,VinylSd,NaN,5,VinylSd,1362,85.0,SBrkr,Lvl,10,TA,...,796,143000,64,1Fam,Gable,50,14172.193820,-57.193820,-166.208560,-195.089240
6,VinylSd,Gd,8,VinylSd,1694,75.0,SBrkr,Lvl,8,TA,...,1694,307000,317,1Fam,Gable,20,10301.210060,-217.210060,-48.917290,-133.983120
7,HdBoard,TA,7,HdBoard,2090,NaN,SBrkr,Lvl,11,TA,...,1107,200000,216,1Fam,Gable,60,10547.823910,-165.823910,-207.336550,30.690990
8,Wd Shng,TA,7,BrkFace,1774,51.0,FuseF,Lvl,4,TA,...,1022,129900,952,1Fam,Gable,50,5902.686193,217.313807,153.416701,215.549815
9,MetalSd,TA,5,MetalSd,1077,50.0,SBrkr,Lvl,1,TA,...,1077,118000,140,2fmCon,Gable,190,7324.456009,95.543991,280.075430,238.067704


...and build a new project with the collinearity removed.

In [18]:
# and build a new project with the reshaped data
print('Creating project...')
dr_proj_nocoll = dr.Project.create(new_model_data, project_name=CREATE_PROJ_NAME + '_coll. removed')

print('Setting target variable...')
dr_proj_nocoll.set_worker_count(MAX_WORKERS)
dr_proj_nocoll.set_target(target=TARGET_FEAT)

dr_proj_nocoll

Creating project...
Setting target variable...


Project(ZPCN - collinear feature impact demo_coll. removed)

Finally, we open that project's leaderboard in a browser window.

In [19]:
dr_proj_nocoll.open_leaderboard_browser()

True